### Ролевая модель, сводный отчет (списки доступа по всем ролям)

In [ ]:
pip install pivottablejs pandas

In [ ]:
from ipywidgets import VBox, Password, Text
token = Password(description='Token:', placeholder='Введите токен') 
display(VBox([token]))

In [ ]:
from sqlalchemy import create_engine, text, StaticPool
from pandas import DataFrame, pivot_table
from pivottablejs import pivot_ui

pgEn = create_engine(f'postgresql+psycopg://{token.value}',poolclass=StaticPool)

pgConnection = pgEn.connect()

resultSet = pgConnection.execute(text(
                                        ''' 
                                        select *
                                        from dbo.eligible_links el LEFT outer join dbo.probes p on (el.link_id = p.link_id and el.role_id =p.role_id and  NOW() - P.created_ts  < INTERVAL '300 minute')
                                        /* where service_name like 'Kafka%' or service_name like 'CDB%' or service_name like 'Swagger%' */
                                        ''')
                                )


#resultSet = pgConnection.execute(text(
#                                        ''' 
#                                        select *
#                                        from dbo.eligible_links 
#                                        /* WHERE service_name like 'Kafka%' or service_name like 'CDB%' or service_name like 'Swagger%' */
#                                        ''')
#                                )







df = DataFrame(resultSet.fetchall())
pgEn.dispose() 

# pivot_table(df, index=['service_name'], columns=['role_name'],sort=True, margins=True,aggfunc={"url":"count", "latency_ns":"min"},values=['url','latency_ns'],fill_value=0)

pivot_table(df, index=['service_name','url'], columns=['role_name'],sort=True, margins=False,aggfunc={"latency_ns":"min"},values=['latency_ns'])


In [ ]:
pivot_table(df, index=['service_name','url'], columns=['role_name'],sort=True, margins=False,aggfunc={"latency_ns":"max"},values=['latency_ns'])

In [ ]:
pivot_ui(df ,outfile_path='pivottablejs7.html')